In [1]:
import pandas as pd
import numpy as np
import random
import os
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import *
from xgboost import *
from catboost import *
from mlxtend.classifier import StackingCVClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

import optuna
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [2]:
train = pd.read_csv('./data/train.csv', encoding='utf-8')
test = pd.read_csv('./data/test.csv', encoding='utf-8')
info = pd.read_csv('./data/snp_info.csv', encoding='utf-8')

In [3]:
train.shape, test.shape

((262, 21), (175, 20))

In [4]:
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


In [5]:
train = train.drop(columns=['id'])
cols = train.columns.tolist()

### EDA

In [6]:
# prof = train.profile_report()

In [7]:
# prof

### 전처리

In [8]:
for i in cols:
    print('columns:',i, train[i].unique())

columns: father [0]
columns: mother [0]
columns: gender [0]
columns: trait [2 1]
columns: SNP_01 ['G G' 'A G' 'A A']
columns: SNP_02 ['A G' 'G G' 'A A']
columns: SNP_03 ['A A' 'C A' 'C C']
columns: SNP_04 ['G A' 'A A' 'G G']
columns: SNP_05 ['C A' 'A A' 'C C']
columns: SNP_06 ['A A' 'A G' 'G G']
columns: SNP_07 ['A A' 'G G' 'G A']
columns: SNP_08 ['G G' 'G A' 'A A']
columns: SNP_09 ['A A' 'G A' 'G G']
columns: SNP_10 ['G G' 'A G' 'A A']
columns: SNP_11 ['A G' 'A A' 'G G']
columns: SNP_12 ['A A' 'G A' 'G G']
columns: SNP_13 ['A A' 'G G' 'A G']
columns: SNP_14 ['A A' 'C C' 'C A']
columns: SNP_15 ['A A' 'G G' 'G A']
columns: class ['B' 'C' 'A']


In [9]:
# SNP 카테고리 확인

snp_list = []
for i in cols[4:-1]:
    for j in range(2):
        snp_list.append([k for k in train[i].unique().tolist()][j])

print(set(snp_list)) # 6개 카테고리

{'A G', 'A A', 'G G', 'C C', 'G A', 'C A'}


In [10]:
# SNP 카테고리는 총 6개이지만 SNP별 카테고리는 3개씩
# SNP별 카테고리를 다시 분리하여 label encoding 적용

# father, mother, gender 열 모두 0만 존재하므로 각 데이터에서 제거

test_id = test[['id']] # test ID 저장
target = train['class']

train = train.iloc[:, 3:-1]
test = test.iloc[:, 4:]

train.shape, test.shape, target.shape

((262, 16), (175, 16), (262,))

In [11]:
train_snp = train.iloc[:, 1:]
train_snp

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A
1,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A
2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A
3,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G
4,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A
258,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A
259,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G
260,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G


In [12]:
df = pd.DataFrame(data=np.zeros((262,30)), columns=list(range(1, 31)))
df

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
for idx, i in enumerate(train_snp.columns.tolist()):
    for j in range(train_snp.shape[0]):
        df.iloc[j, (idx*2)] = train_snp[i][j][0]
        df.iloc[j, (((idx+1)*2)-1)] = train_snp[i][j][2]

df

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,G,G,A,G,A,A,G,A,C,A,...,A,G,A,A,A,A,A,A,A,A
1,A,G,A,G,C,A,A,A,A,A,...,A,A,G,A,G,G,A,A,A,A
2,G,G,G,G,A,A,G,A,C,C,...,A,A,A,A,A,A,A,A,A,A
3,A,A,G,G,A,A,G,A,A,A,...,G,G,G,G,G,G,A,A,G,G
4,G,G,G,G,C,C,A,A,C,C,...,A,A,A,A,A,G,A,A,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,A,G,A,G,A,A,G,A,C,C,...,A,G,G,A,A,A,A,A,A,A
258,G,G,A,A,C,A,A,A,A,A,...,A,G,A,A,A,G,A,A,G,A
259,A,G,G,G,A,A,G,A,A,A,...,G,G,G,G,G,G,C,A,G,G
260,A,A,G,G,A,A,G,A,A,A,...,A,G,G,A,G,G,C,A,G,G


In [14]:
df.columns = ['SNP_' + str(i+1) for i in range(30)]
df

,SNP_1,SNP_2,SNP_3,SNP_4,SNP_5,SNP_6,SNP_7,SNP_8,SNP_9,SNP_10,...,SNP_21,SNP_22,SNP_23,SNP_24,SNP_25,SNP_26,SNP_27,SNP_28,SNP_29,SNP_30
0,G,G,A,G,A,A,G,A,C,A,...,A,G,A,A,A,A,A,A,A,A
1,A,G,A,G,C,A,A,A,A,A,...,A,A,G,A,G,G,A,A,A,A
2,G,G,G,G,A,A,G,A,C,C,...,A,A,A,A,A,A,A,A,A,A
3,A,A,G,G,A,A,G,A,A,A,...,G,G,G,G,G,G,A,A,G,G
4,G,G,G,G,C,C,A,A,C,C,...,A,A,A,A,A,G,A,A,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,A,G,A,G,A,A,G,A,C,C,...,A,G,G,A,A,A,A,A,A,A
258,G,G,A,A,C,A,A,A,A,A,...,A,G,A,A,A,G,A,A,G,A
259,A,G,G,G,A,A,G,A,A,A,...,G,G,G,G,G,G,C,A,G,G
260,A,A,G,G,A,A,G,A,A,A,...,A,G,G,A,G,G,C,A,G,G


In [15]:
train_snp = df

In [16]:
test_snp = test.iloc[:, 1:]
test_snp

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


In [17]:
df = pd.DataFrame(data=np.zeros((175,30)), columns=list(range(1, 31)))
df

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
for idx, i in enumerate(test_snp.columns.tolist()):
    for j in range(test_snp.shape[0]):
        df.iloc[j, (idx*2)] = test_snp[i][j][0]
        df.iloc[j, (((idx+1)*2)-1)] = test_snp[i][j][2]

df

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,A,G,G,G,A,A,G,A,A,A,...,A,G,G,A,G,G,C,A,G,A
1,G,G,A,G,C,C,G,G,C,C,...,A,G,A,A,A,A,A,A,A,A
2,G,G,A,G,A,A,A,A,C,A,...,A,A,G,A,G,G,A,A,G,G
3,G,G,A,G,C,A,A,A,C,C,...,A,A,G,A,A,G,A,A,A,A
4,A,A,G,G,A,A,G,G,A,A,...,G,G,G,A,G,G,A,A,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,A,G,G,G,C,C,A,A,C,A,...,G,G,A,A,A,A,A,A,G,A
171,G,G,A,A,A,A,A,A,C,A,...,A,A,A,A,A,G,A,A,G,A
172,G,G,A,A,A,A,A,A,C,A,...,A,G,A,A,A,G,A,A,G,G
173,A,G,G,G,C,A,G,A,C,C,...,A,G,A,A,A,A,A,A,A,A


In [19]:
df.columns = ['SNP_' + str(i+1) for i in range(30)]
df

,SNP_1,SNP_2,SNP_3,SNP_4,SNP_5,SNP_6,SNP_7,SNP_8,SNP_9,SNP_10,...,SNP_21,SNP_22,SNP_23,SNP_24,SNP_25,SNP_26,SNP_27,SNP_28,SNP_29,SNP_30
0,A,G,G,G,A,A,G,A,A,A,...,A,G,G,A,G,G,C,A,G,A
1,G,G,A,G,C,C,G,G,C,C,...,A,G,A,A,A,A,A,A,A,A
2,G,G,A,G,A,A,A,A,C,A,...,A,A,G,A,G,G,A,A,G,G
3,G,G,A,G,C,A,A,A,C,C,...,A,A,G,A,A,G,A,A,A,A
4,A,A,G,G,A,A,G,G,A,A,...,G,G,G,A,G,G,A,A,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,A,G,G,G,C,C,A,A,C,A,...,G,G,A,A,A,A,A,A,G,A
171,G,G,A,A,A,A,A,A,C,A,...,A,A,A,A,A,G,A,A,G,A
172,G,G,A,A,A,A,A,A,C,A,...,A,G,A,A,A,G,A,A,G,G
173,A,G,G,G,C,A,G,A,C,C,...,A,G,A,A,A,A,A,A,A,A


In [20]:
test_snp = df

In [21]:
train = pd.concat([train.trait, train_snp], axis=1)
test = pd.concat([test.trait, test_snp], axis=1)

In [22]:
train_oh = pd.get_dummies(train, drop_first=True)
test_oh = pd.get_dummies(test, drop_first=True)

In [23]:
target.unique()

array(['B', 'C', 'A'], dtype=object)

In [28]:
# Target - Label Encoding

for i, j in zip(range(3), ['A', 'B', 'C']):
    target = target.replace(j, int(i))
    
target

0      1
1      2
2      1
3      0
4      2
      ..
257    1
258    2
259    0
260    0
261    1
Name: class, Length: 262, dtype: int64

In [29]:
# 불균형 해결 : SMOTE로 데이터 생성
smote = SMOTE(random_state=2023, sampling_strategy={0:138, 1:228, 2:158})

train_os, target_os = smote.fit_resample(train_oh, target)

In [31]:
from sklearn.model_selection import train_test_split

tr, tr_val, tar, tar_val = train_test_split(train_os, target_os, stratify=target_os, random_state=2023, test_size=0.25)

In [32]:
# CatBoost 및 optuna Tuning

def cattuner(trial):
    param = {
                #'iterations' : 10000, replaced by early stopping
                'eval_metric': 'TotalF1',
                'use_best_model': True,
                'random_seed' : 2023,
                'learning_rate' :trial.suggest_loguniform('learning_rate', 0.01, 0.3),
                "depth": trial.suggest_int("depth", 1, 15),
                'l2_leaf_reg' :trial.suggest_loguniform('l2_leaf_reg', 1e-8, 20),
                'random_strength' : trial.suggest_loguniform('random_strength', 1, 50),
                'grow_policy':trial.suggest_categorical ('grow_policy', ['Lossguide','SymmetricTree']),
                'max_bin': trial.suggest_int("max_bin", 20, 500),
                'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 100),
                "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"])
            }
    
    if param['grow_policy'] == 'Lossguide':
        param['max_leaves']:trial.suggest_int('max_leaves', 1, 100)
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    model = CatBoostClassifier(**param)
    model.fit(tr, tar, eval_set=[(tr_val, tar_val)], verbose=0, early_stopping_rounds=100)
    
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(cattuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_cb = CatBoostClassifier(**study.best_params)

# cb_params = {'learning_rate': 0.1511954206639055, 'depth': 14, 'l2_leaf_reg': 1.6396467267070022e-07, 'random_strength': 25.29377707945203, 'grow_policy': 'SymmetricTree', 'max_bin': 345, 'min_data_in_leaf': 61, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.1251654730087735}


[I 2023-01-09 22:01:34,020] A new study created in memory with name: no-name-d8de0190-f969-43f1-bf35-922a191befda
[I 2023-01-09 22:01:35,026] Trial 0 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.011788393641065868, 'depth': 1, 'l2_leaf_reg': 0.5085848775765605, 'random_strength': 3.1580994689862183, 'grow_policy': 'SymmetricTree', 'max_bin': 493, 'min_data_in_leaf': 3, 'bootstrap_type': 'Bernoulli', 'subsample': 0.5860018703004174}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:01:35,460] Trial 1 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.06890294718809345, 'depth': 5, 'l2_leaf_reg': 1.930363458775798e-05, 'random_strength': 2.0801107088526964, 'grow_policy': 'Lossguide', 'max_bin': 116, 'min_data_in_leaf': 53, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.300952448517589}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:02:00,145] Trial 2 finished with value: 0.9784659784659785

[I 2023-01-09 22:03:17,556] Trial 20 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.09005350311027875, 'depth': 13, 'l2_leaf_reg': 9.890102656106884e-07, 'random_strength': 21.084211300861785, 'grow_policy': 'SymmetricTree', 'max_bin': 74, 'min_data_in_leaf': 24, 'bootstrap_type': 'Bernoulli', 'subsample': 0.43587735403454564}. Best is trial 15 with value: 0.9928820399926619.
[I 2023-01-09 22:03:18,023] Trial 21 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.024698438238663873, 'depth': 3, 'l2_leaf_reg': 0.03180112576788002, 'random_strength': 9.535684792873944, 'grow_policy': 'SymmetricTree', 'max_bin': 21, 'min_data_in_leaf': 22, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.3478647338030488}. Best is trial 15 with value: 0.9928820399926619.
[I 2023-01-09 22:03:18,940] Trial 22 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.19011022769029248, 'depth': 7, 'l2_leaf_reg': 0.001751507538472647

[I 2023-01-09 22:04:09,912] Trial 40 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.06576702334485866, 'depth': 10, 'l2_leaf_reg': 0.007033661222623947, 'random_strength': 6.304307864952702, 'grow_policy': 'SymmetricTree', 'max_bin': 158, 'min_data_in_leaf': 45, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6298832955733631}. Best is trial 15 with value: 0.9928820399926619.
[I 2023-01-09 22:04:59,623] Trial 41 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.0922605194191025, 'depth': 15, 'l2_leaf_reg': 2.0836078840923317e-05, 'random_strength': 2.774138557693507, 'grow_policy': 'SymmetricTree', 'max_bin': 344, 'min_data_in_leaf': 99, 'bootstrap_type': 'Bernoulli', 'subsample': 0.4973107460516957}. Best is trial 15 with value: 0.9928820399926619.
[I 2023-01-09 22:05:09,063] Trial 42 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.0702971452615894, 'depth': 12, 'l2_leaf_reg': 0.0002569689223710269, 'rand

Number of finished trials: 50
Best trial: {'learning_rate': 0.17659212611242592, 'depth': 6, 'l2_leaf_reg': 0.0001907584108626437, 'random_strength': 10.251931281074787, 'grow_policy': 'SymmetricTree', 'max_bin': 201, 'min_data_in_leaf': 66, 'bootstrap_type': 'Bernoulli', 'subsample': 0.9658617192798391}


In [33]:
# XGBoost 

def xgbtuner(trial, data=tr, target=tar):
    
    param = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_int('max_depth', 3,30),
        'min_child_weight': trial.suggest_categorical('min_child_weight', [1, 3, 5]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'n_estimators' : trial.suggest_categorical('n_estimators', [100,200,300,400,500]),
        'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
        'random_state': 2023
    }
    model = XGBClassifier(**param)
    
    model.fit(tr, tar)
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(xgbtuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_xgb = XGBClassifier(**study.best_params)

# xgb_params = {'learning_rate': 0.008, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.6, 'colsample_bytree': 0.3, 'n_estimators': 200, 'objective': 'reg:squarederror'}

[I 2023-01-09 22:06:08,165] A new study created in memory with name: no-name-7ca6dd99-6b50-4108-95c4-24212052319e
[I 2023-01-09 22:06:11,800] Trial 0 finished with value: 0.9711567053226288 and parameters: {'learning_rate': 0.017, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 1.0, 'colsample_bytree': 0.9, 'n_estimators': 200, 'objective': 'reg:squarederror'}. Best is trial 0 with value: 0.9711567053226288.
[I 2023-01-09 22:06:12,129] Trial 1 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.01, 'max_depth': 19, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.3, 'n_estimators': 200, 'objective': 'reg:squarederror'}. Best is trial 1 with value: 0.9784659784659785.
[I 2023-01-09 22:06:12,885] Trial 2 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.01, 'max_depth': 28, 'min_child_weight': 5, 'subsample': 1.0, 'colsample_bytree': 0.3, 'n_estimators': 500, 'objective': 'reg:squarederror'}. Best is trial 2 with value: 

[I 2023-01-09 22:06:28,319] Trial 26 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.008, 'max_depth': 23, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.6, 'n_estimators': 400, 'objective': 'reg:squarederror'}. Best is trial 2 with value: 0.9857058650162099.
[I 2023-01-09 22:06:28,515] Trial 27 finished with value: 0.9711567053226288 and parameters: {'learning_rate': 0.01, 'max_depth': 27, 'min_child_weight': 5, 'subsample': 0.6, 'colsample_bytree': 0.4, 'n_estimators': 100, 'objective': 'reg:squarederror'}. Best is trial 2 with value: 0.9857058650162099.
[I 2023-01-09 22:06:29,065] Trial 28 finished with value: 0.9711567053226288 and parameters: {'learning_rate': 0.008, 'max_depth': 24, 'min_child_weight': 3, 'subsample': 0.4, 'colsample_bytree': 0.6, 'n_estimators': 300, 'objective': 'reg:squarederror'}. Best is trial 2 with value: 0.9857058650162099.
[I 2023-01-09 22:06:29,445] Trial 29 finished with value: 0.9644101999633095 and paramete

Number of finished trials: 50
Best trial: {'learning_rate': 0.01, 'max_depth': 28, 'min_child_weight': 5, 'subsample': 1.0, 'colsample_bytree': 0.3, 'n_estimators': 500, 'objective': 'reg:squarederror'}


In [34]:
# LightGBM

def lgbtuner(trial,data=tr,target=tar):
    
    param = {
        'metric': 'multi_error', 
        'random_state': 2023,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100),
        'random_state' : 2023
    }
    model = LGBMClassifier(**param)
    
    model.fit(tr,tar,eval_set=[(tr_val,tar_val)], verbose=0, early_stopping_rounds= 100)
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(lgbtuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_lgb = LGBMClassifier(**study.best_params) 

# lgb_params = {'reg_alpha': 0.16768226852129442, 'reg_lambda': 0.018910145981297394, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.006, 'max_depth': 10, 'num_leaves': 783}

[I 2023-01-09 22:06:41,261] A new study created in memory with name: no-name-8d33e2a8-bd59-4e49-ab1f-45a2d9acc9a9
[I 2023-01-09 22:07:06,830] Trial 0 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.12834951277281972, 'reg_lambda': 0.0025861565438158687, 'colsample_bytree': 1.0, 'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 100, 'num_leaves': 139, 'min_data_per_groups': 54}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:07:07,127] Trial 1 finished with value: 0.9637721755368814 and parameters: {'reg_alpha': 0.001877067542292141, 'reg_lambda': 0.17821781119504967, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.01, 'max_depth': 10, 'num_leaves': 567, 'min_data_per_groups': 86}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:07:07,421] Trial 2 finished with value: 0.9497539497539497 and parameters: {'reg_alpha': 0.6043101537162113, 'reg_lambda': 0.0030990549170323567, 'colsample_bytree': 0.9, 'subsample': 

[I 2023-01-09 22:07:13,910] Trial 23 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.0010439080885548608, 'reg_lambda': 1.1136047421289204, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 0.017, 'max_depth': 10, 'num_leaves': 176, 'min_data_per_groups': 43}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:07:14,146] Trial 24 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 1.5737005282667986, 'reg_lambda': 0.46683264978344396, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.017, 'max_depth': 100, 'num_leaves': 95, 'min_data_per_groups': 63}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:07:14,549] Trial 25 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.006371462800161456, 'reg_lambda': 0.05402682554996841, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.014, 'max_depth': 100, 'num_leaves': 253, 'min_data_per_groups': 59}. Best is trial 0 with

[I 2023-01-09 22:07:21,049] Trial 47 finished with value: 0.9637721755368814 and parameters: {'reg_alpha': 1.3601465374980783, 'reg_lambda': 0.25011012354137846, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.008, 'max_depth': 20, 'num_leaves': 498, 'min_data_per_groups': 36}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:07:21,426] Trial 48 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.002064118536489595, 'reg_lambda': 0.34292316226667635, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.014, 'max_depth': 100, 'num_leaves': 246, 'min_data_per_groups': 87}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-09 22:07:21,680] Trial 49 finished with value: 0.9637721755368814 and parameters: {'reg_alpha': 0.8263566021685783, 'reg_lambda': 0.10109619385747735, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.006, 'max_depth': 10, 'num_leaves': 152, 'min_data_per_groups': 50}. Best is trial 0 with v

Number of finished trials: 50
Best trial: {'reg_alpha': 0.12834951277281972, 'reg_lambda': 0.0025861565438158687, 'colsample_bytree': 1.0, 'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 100, 'num_leaves': 139, 'min_data_per_groups': 54}


In [37]:
# KNN

from sklearn.model_selection import cross_val_score, RepeatedKFold

rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=2023)

prs = []

for i in range(2, 25):
    knn = KNeighborsClassifier(n_neighbors=i)
    score = cross_val_score(knn, train_os, target_os, scoring='f1_macro', cv=rkf, n_jobs=-1)
    print('n_neighbors :', i, score.mean())
    prs.append([i, score.mean()])

np.argmax([prs[i][1] for i in range(len(prs))])

# n_neighbors = 3

n_neighbors : 2 0.9677321477877318
n_neighbors : 3 0.979202037234566
n_neighbors : 4 0.9700670654184168
n_neighbors : 5 0.9751947312742424
n_neighbors : 6 0.9654444926692406
n_neighbors : 7 0.9709689073091977
n_neighbors : 8 0.9653069811790439
n_neighbors : 9 0.9681076016768437
n_neighbors : 10 0.9629039865221074
n_neighbors : 11 0.9686930661507662
n_neighbors : 12 0.9652703873922386
n_neighbors : 13 0.9654360440001233
n_neighbors : 14 0.9629067162315675
n_neighbors : 15 0.9644897299241706
n_neighbors : 16 0.9573344563078016
n_neighbors : 17 0.9621667602426781
n_neighbors : 18 0.959552424756076
n_neighbors : 19 0.9633578503664711
n_neighbors : 20 0.9605620339006989
n_neighbors : 21 0.9640637603579387
n_neighbors : 22 0.9613202858502851
n_neighbors : 23 0.9644925584295159
n_neighbors : 24 0.9610658434889727


1

In [38]:
tuned_knn = KNeighborsClassifier(n_neighbors=3)

from tqdm import tqdm

for i in tqdm(range(4)):
    estimators = [('xgb', tuned_xgb), ('lgb', tuned_lgb), ('cb', tuned_cb), ('knn', tuned_knn)]
    fin_estimator = [tuned_xgb, tuned_lgb, tuned_cb, tuned_knn]
    
    est_list = estimators.pop(i)
    fin_est = fin_estimator.pop(i)
    
    stacks = StackingClassifier(estimators=estimators, final_estimator=fin_est, stack_method='predict_proba', cv=10)
    stacks.fit(tr, tar)
    
    pred = stacks.predict(tr_val)
    
    rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=2023)
    score = cross_val_score(stacks, train_os, target_os, scoring='f1_macro', cv=rkf, n_jobs=-1)
    
    print(f'Final Estimator : {fin_est} Model Score :', score.mean())    

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0:	learn: 0.9259850	total: 2.68ms	remaining: 2.67s
1:	learn: 0.7734152	total: 9.09ms	remaining: 4.54s
2:	learn: 0.6577809	total: 12.1ms	remaining: 4s
3:	learn: 0.5484998	total: 15ms	remaining: 3.74s
4:	learn: 0.4751479	total: 18ms	remaining: 3.58s
5:	learn: 0.4118690	total: 21.3ms	remaining: 3.54s
6:	learn: 0.3640924	total: 24.2ms	remaining: 3.43s
7:	learn: 0.3554246	total: 25ms	remaining: 3.1s
8:	learn: 0.3232654	total: 27.7ms	remaining: 3.05s
9:	learn: 0.2899143	total: 29.7ms	remaining: 2.94s
10:	learn: 0.2559608	total: 32.7ms	remaining: 2.94s
11:	learn: 0.2372058	total: 34.6ms	remaining: 2.85s
12:	learn: 0.2166944	total: 37.5ms	remaining: 2.85s
13:	learn: 0.1965732	total: 40.3ms	remaining: 2.83s
14:	learn: 0.1795610	total: 43ms	remaining: 2.82s
15:	learn: 0.1593583	total: 46.3ms	remaining: 2.85s
16:	learn: 0.1481860	total: 49.1ms	remaining: 2.84s
17:	learn: 0.1362341	total: 50.7ms	remaining: 2.77s
18:	learn: 0.1216828	total

204:	learn: 0.0000027	total: 587ms	remaining: 2.27s
205:	learn: 0.0000027	total: 590ms	remaining: 2.27s
206:	learn: 0.0000026	total: 593ms	remaining: 2.27s
207:	learn: 0.0000026	total: 596ms	remaining: 2.27s
208:	learn: 0.0000025	total: 599ms	remaining: 2.27s
209:	learn: 0.0000025	total: 602ms	remaining: 2.26s
210:	learn: 0.0000024	total: 604ms	remaining: 2.26s
211:	learn: 0.0000024	total: 607ms	remaining: 2.26s
212:	learn: 0.0000024	total: 610ms	remaining: 2.25s
213:	learn: 0.0000023	total: 613ms	remaining: 2.25s
214:	learn: 0.0000023	total: 616ms	remaining: 2.25s
215:	learn: 0.0000022	total: 619ms	remaining: 2.24s
216:	learn: 0.0000022	total: 621ms	remaining: 2.24s
217:	learn: 0.0000021	total: 624ms	remaining: 2.24s
218:	learn: 0.0000021	total: 627ms	remaining: 2.24s
219:	learn: 0.0000021	total: 630ms	remaining: 2.23s
220:	learn: 0.0000020	total: 633ms	remaining: 2.23s
221:	learn: 0.0000020	total: 636ms	remaining: 2.23s
222:	learn: 0.0000019	total: 639ms	remaining: 2.23s
223:	learn: 

378:	learn: 0.0000005	total: 1.14s	remaining: 1.88s
379:	learn: 0.0000005	total: 1.15s	remaining: 1.87s
380:	learn: 0.0000005	total: 1.15s	remaining: 1.87s
381:	learn: 0.0000005	total: 1.15s	remaining: 1.87s
382:	learn: 0.0000005	total: 1.16s	remaining: 1.86s
383:	learn: 0.0000005	total: 1.16s	remaining: 1.86s
384:	learn: 0.0000005	total: 1.16s	remaining: 1.86s
385:	learn: 0.0000005	total: 1.17s	remaining: 1.85s
386:	learn: 0.0000005	total: 1.17s	remaining: 1.85s
387:	learn: 0.0000005	total: 1.17s	remaining: 1.85s
388:	learn: 0.0000005	total: 1.17s	remaining: 1.84s
389:	learn: 0.0000005	total: 1.18s	remaining: 1.84s
390:	learn: 0.0000005	total: 1.18s	remaining: 1.84s
391:	learn: 0.0000005	total: 1.18s	remaining: 1.83s
392:	learn: 0.0000005	total: 1.19s	remaining: 1.83s
393:	learn: 0.0000005	total: 1.19s	remaining: 1.83s
394:	learn: 0.0000005	total: 1.19s	remaining: 1.83s
395:	learn: 0.0000005	total: 1.2s	remaining: 1.82s
396:	learn: 0.0000005	total: 1.2s	remaining: 1.82s
397:	learn: 0.

577:	learn: 0.0000003	total: 1.73s	remaining: 1.26s
578:	learn: 0.0000003	total: 1.73s	remaining: 1.26s
579:	learn: 0.0000002	total: 1.73s	remaining: 1.25s
580:	learn: 0.0000002	total: 1.74s	remaining: 1.25s
581:	learn: 0.0000002	total: 1.74s	remaining: 1.25s
582:	learn: 0.0000002	total: 1.74s	remaining: 1.25s
583:	learn: 0.0000002	total: 1.75s	remaining: 1.24s
584:	learn: 0.0000002	total: 1.75s	remaining: 1.24s
585:	learn: 0.0000002	total: 1.75s	remaining: 1.24s
586:	learn: 0.0000002	total: 1.75s	remaining: 1.24s
587:	learn: 0.0000002	total: 1.76s	remaining: 1.23s
588:	learn: 0.0000002	total: 1.76s	remaining: 1.23s
589:	learn: 0.0000002	total: 1.76s	remaining: 1.23s
590:	learn: 0.0000002	total: 1.77s	remaining: 1.22s
591:	learn: 0.0000002	total: 1.77s	remaining: 1.22s
592:	learn: 0.0000002	total: 1.77s	remaining: 1.22s
593:	learn: 0.0000002	total: 1.77s	remaining: 1.21s
594:	learn: 0.0000002	total: 1.78s	remaining: 1.21s
595:	learn: 0.0000002	total: 1.78s	remaining: 1.21s
596:	learn: 

...

407:	learn: 0.0000004	total: 1.17s	remaining: 1.69s
408:	learn: 0.0000004	total: 1.17s	remaining: 1.69s
409:	learn: 0.0000004	total: 1.17s	remaining: 1.69s
410:	learn: 0.0000004	total: 1.18s	remaining: 1.68s
411:	learn: 0.0000004	total: 1.18s	remaining: 1.68s
412:	learn: 0.0000004	total: 1.18s	remaining: 1.68s
413:	learn: 0.0000004	total: 1.18s	remaining: 1.68s
414:	learn: 0.0000004	total: 1.19s	remaining: 1.67s
415:	learn: 0.0000004	total: 1.19s	remaining: 1.67s
416:	learn: 0.0000004	total: 1.19s	remaining: 1.67s
417:	learn: 0.0000004	total: 1.2s	remaining: 1.66s
418:	learn: 0.0000004	total: 1.2s	remaining: 1.66s
419:	learn: 0.0000004	total: 1.2s	remaining: 1.66s
420:	learn: 0.0000004	total: 1.2s	remaining: 1.66s
421:	learn: 0.0000004	total: 1.21s	remaining: 1.65s
422:	learn: 0.0000004	total: 1.21s	remaining: 1.65s
423:	learn: 0.0000004	total: 1.21s	remaining: 1.65s
424:	learn: 0.0000004	total: 1.22s	remaining: 1.65s
425:	learn: 0.0000004	total: 1.22s	remaining: 1.64s
426:	learn: 0.00

614:	learn: 0.0000002	total: 1.75s	remaining: 1.09s
615:	learn: 0.0000002	total: 1.75s	remaining: 1.09s
616:	learn: 0.0000002	total: 1.75s	remaining: 1.09s
617:	learn: 0.0000002	total: 1.76s	remaining: 1.09s
618:	learn: 0.0000002	total: 1.76s	remaining: 1.08s
619:	learn: 0.0000002	total: 1.76s	remaining: 1.08s
620:	learn: 0.0000002	total: 1.77s	remaining: 1.08s
621:	learn: 0.0000002	total: 1.77s	remaining: 1.07s
622:	learn: 0.0000002	total: 1.77s	remaining: 1.07s
623:	learn: 0.0000002	total: 1.77s	remaining: 1.07s
624:	learn: 0.0000002	total: 1.78s	remaining: 1.07s
625:	learn: 0.0000002	total: 1.78s	remaining: 1.06s
626:	learn: 0.0000002	total: 1.78s	remaining: 1.06s
627:	learn: 0.0000002	total: 1.79s	remaining: 1.06s
628:	learn: 0.0000002	total: 1.79s	remaining: 1.06s
629:	learn: 0.0000002	total: 1.79s	remaining: 1.05s
630:	learn: 0.0000002	total: 1.8s	remaining: 1.05s
631:	learn: 0.0000002	total: 1.8s	remaining: 1.05s
632:	learn: 0.0000002	total: 1.8s	remaining: 1.04s
633:	learn: 0.0

820:	learn: 0.0000002	total: 2.33s	remaining: 509ms
821:	learn: 0.0000002	total: 2.34s	remaining: 506ms
822:	learn: 0.0000002	total: 2.34s	remaining: 503ms
823:	learn: 0.0000002	total: 2.34s	remaining: 500ms
824:	learn: 0.0000002	total: 2.34s	remaining: 497ms
825:	learn: 0.0000002	total: 2.35s	remaining: 495ms
826:	learn: 0.0000002	total: 2.35s	remaining: 492ms
827:	learn: 0.0000002	total: 2.35s	remaining: 489ms
828:	learn: 0.0000002	total: 2.35s	remaining: 486ms
829:	learn: 0.0000002	total: 2.36s	remaining: 483ms
830:	learn: 0.0000002	total: 2.36s	remaining: 480ms
831:	learn: 0.0000002	total: 2.36s	remaining: 477ms
832:	learn: 0.0000002	total: 2.37s	remaining: 474ms
833:	learn: 0.0000002	total: 2.37s	remaining: 472ms
834:	learn: 0.0000002	total: 2.37s	remaining: 469ms
835:	learn: 0.0000002	total: 2.37s	remaining: 466ms
836:	learn: 0.0000002	total: 2.38s	remaining: 463ms
837:	learn: 0.0000002	total: 2.38s	remaining: 460ms
838:	learn: 0.0000002	total: 2.38s	remaining: 457ms
839:	learn: 

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [23:11<00:00, 347.91s/it]

Final Estimator : KNeighborsClassifier(n_neighbors=3) Model Score : 0.9789220605611517


In [39]:
# Final Estimator - XGBoost

estimators = [('xgb', tuned_xgb), ('cb', tuned_cb), ('knn', tuned_knn)]
fin_estimator = tuned_lgb
    
# StackingClassification
stacks = StackingClassifier(estimators=estimators, final_estimator=fin_estimator, stack_method='predict_proba', cv=10)
stacks.fit(train_os, target_os)
    
preds = stacks.predict(test_oh)

0:	learn: 0.9667472	total: 1.26ms	remaining: 1.26s
1:	learn: 0.7921051	total: 4.31ms	remaining: 2.15s
2:	learn: 0.6468288	total: 7.79ms	remaining: 2.59s
3:	learn: 0.5510759	total: 11.3ms	remaining: 2.81s
4:	learn: 0.4920213	total: 14.2ms	remaining: 2.83s
5:	learn: 0.4474626	total: 16.3ms	remaining: 2.69s
6:	learn: 0.4046790	total: 19.3ms	remaining: 2.74s
7:	learn: 0.3721889	total: 20.9ms	remaining: 2.6s
8:	learn: 0.3286890	total: 23.8ms	remaining: 2.62s
9:	learn: 0.2885756	total: 27.1ms	remaining: 2.68s
10:	learn: 0.2660270	total: 30.3ms	remaining: 2.72s
11:	learn: 0.2402856	total: 33.5ms	remaining: 2.76s
12:	learn: 0.2225972	total: 34.7ms	remaining: 2.63s
13:	learn: 0.2033139	total: 37.7ms	remaining: 2.66s
14:	learn: 0.1888124	total: 41ms	remaining: 2.69s
15:	learn: 0.1759976	total: 44.3ms	remaining: 2.72s
16:	learn: 0.1619623	total: 47.5ms	remaining: 2.75s
17:	learn: 0.1568574	total: 50.8ms	remaining: 2.77s
18:	learn: 0.1471186	total: 53.9ms	remaining: 2.78s
19:	learn: 0.1426048	tota

192:	learn: 0.0000059	total: 580ms	remaining: 2.42s
193:	learn: 0.0000058	total: 583ms	remaining: 2.42s
194:	learn: 0.0000056	total: 587ms	remaining: 2.42s
195:	learn: 0.0000055	total: 590ms	remaining: 2.42s
196:	learn: 0.0000054	total: 593ms	remaining: 2.42s
197:	learn: 0.0000052	total: 597ms	remaining: 2.42s
198:	learn: 0.0000051	total: 600ms	remaining: 2.41s
199:	learn: 0.0000050	total: 603ms	remaining: 2.41s
200:	learn: 0.0000048	total: 606ms	remaining: 2.41s
201:	learn: 0.0000047	total: 610ms	remaining: 2.41s
202:	learn: 0.0000046	total: 613ms	remaining: 2.4s
203:	learn: 0.0000045	total: 616ms	remaining: 2.4s
204:	learn: 0.0000044	total: 619ms	remaining: 2.4s
205:	learn: 0.0000043	total: 622ms	remaining: 2.4s
206:	learn: 0.0000042	total: 625ms	remaining: 2.39s
207:	learn: 0.0000040	total: 628ms	remaining: 2.39s
208:	learn: 0.0000040	total: 631ms	remaining: 2.39s
209:	learn: 0.0000039	total: 634ms	remaining: 2.38s
210:	learn: 0.0000038	total: 638ms	remaining: 2.38s
211:	learn: 0.00

378:	learn: 0.0000006	total: 1.16s	remaining: 1.9s
379:	learn: 0.0000006	total: 1.16s	remaining: 1.9s
380:	learn: 0.0000006	total: 1.17s	remaining: 1.89s
381:	learn: 0.0000006	total: 1.17s	remaining: 1.89s
382:	learn: 0.0000006	total: 1.17s	remaining: 1.89s
383:	learn: 0.0000006	total: 1.17s	remaining: 1.88s
384:	learn: 0.0000006	total: 1.18s	remaining: 1.88s
385:	learn: 0.0000006	total: 1.18s	remaining: 1.88s
386:	learn: 0.0000006	total: 1.18s	remaining: 1.88s
387:	learn: 0.0000005	total: 1.19s	remaining: 1.87s
388:	learn: 0.0000005	total: 1.19s	remaining: 1.87s
389:	learn: 0.0000005	total: 1.19s	remaining: 1.87s
390:	learn: 0.0000005	total: 1.2s	remaining: 1.86s
391:	learn: 0.0000005	total: 1.2s	remaining: 1.86s
392:	learn: 0.0000005	total: 1.2s	remaining: 1.86s
393:	learn: 0.0000005	total: 1.2s	remaining: 1.85s
394:	learn: 0.0000005	total: 1.21s	remaining: 1.85s
395:	learn: 0.0000005	total: 1.21s	remaining: 1.85s
396:	learn: 0.0000005	total: 1.21s	remaining: 1.84s
397:	learn: 0.0000

562:	learn: 0.0000003	total: 1.74s	remaining: 1.35s
563:	learn: 0.0000003	total: 1.74s	remaining: 1.34s
564:	learn: 0.0000003	total: 1.74s	remaining: 1.34s
565:	learn: 0.0000003	total: 1.75s	remaining: 1.34s
566:	learn: 0.0000003	total: 1.75s	remaining: 1.33s
567:	learn: 0.0000003	total: 1.75s	remaining: 1.33s
568:	learn: 0.0000003	total: 1.76s	remaining: 1.33s
569:	learn: 0.0000003	total: 1.76s	remaining: 1.33s
570:	learn: 0.0000003	total: 1.76s	remaining: 1.32s
571:	learn: 0.0000003	total: 1.76s	remaining: 1.32s
572:	learn: 0.0000003	total: 1.77s	remaining: 1.32s
573:	learn: 0.0000003	total: 1.77s	remaining: 1.31s
574:	learn: 0.0000003	total: 1.77s	remaining: 1.31s
575:	learn: 0.0000003	total: 1.78s	remaining: 1.31s
576:	learn: 0.0000003	total: 1.78s	remaining: 1.31s
577:	learn: 0.0000003	total: 1.78s	remaining: 1.3s
578:	learn: 0.0000003	total: 1.79s	remaining: 1.3s
579:	learn: 0.0000003	total: 1.79s	remaining: 1.3s
580:	learn: 0.0000003	total: 1.79s	remaining: 1.29s
581:	learn: 0.0

736:	learn: 0.0000002	total: 2.3s	remaining: 821ms
737:	learn: 0.0000002	total: 2.3s	remaining: 818ms
738:	learn: 0.0000002	total: 2.31s	remaining: 815ms
739:	learn: 0.0000002	total: 2.31s	remaining: 812ms
740:	learn: 0.0000002	total: 2.31s	remaining: 809ms
741:	learn: 0.0000002	total: 2.32s	remaining: 806ms
742:	learn: 0.0000002	total: 2.32s	remaining: 803ms
743:	learn: 0.0000002	total: 2.32s	remaining: 799ms
744:	learn: 0.0000002	total: 2.33s	remaining: 796ms
745:	learn: 0.0000002	total: 2.33s	remaining: 793ms
746:	learn: 0.0000002	total: 2.33s	remaining: 790ms
747:	learn: 0.0000002	total: 2.34s	remaining: 787ms
748:	learn: 0.0000002	total: 2.34s	remaining: 784ms
749:	learn: 0.0000002	total: 2.34s	remaining: 781ms
750:	learn: 0.0000002	total: 2.35s	remaining: 778ms
751:	learn: 0.0000002	total: 2.35s	remaining: 775ms
752:	learn: 0.0000002	total: 2.35s	remaining: 772ms
753:	learn: 0.0000002	total: 2.36s	remaining: 769ms
754:	learn: 0.0000002	total: 2.36s	remaining: 766ms
755:	learn: 0.

...

195:	learn: 0.0000067	total: 583ms	remaining: 2.39s
196:	learn: 0.0000066	total: 586ms	remaining: 2.39s
197:	learn: 0.0000063	total: 589ms	remaining: 2.39s
198:	learn: 0.0000061	total: 592ms	remaining: 2.38s
199:	learn: 0.0000059	total: 595ms	remaining: 2.38s
200:	learn: 0.0000057	total: 599ms	remaining: 2.38s
201:	learn: 0.0000055	total: 602ms	remaining: 2.38s
202:	learn: 0.0000053	total: 605ms	remaining: 2.37s
203:	learn: 0.0000051	total: 608ms	remaining: 2.37s
204:	learn: 0.0000050	total: 611ms	remaining: 2.37s
205:	learn: 0.0000048	total: 614ms	remaining: 2.36s
206:	learn: 0.0000047	total: 617ms	remaining: 2.36s
207:	learn: 0.0000045	total: 620ms	remaining: 2.36s
208:	learn: 0.0000044	total: 622ms	remaining: 2.35s
209:	learn: 0.0000044	total: 625ms	remaining: 2.35s
210:	learn: 0.0000043	total: 628ms	remaining: 2.35s
211:	learn: 0.0000042	total: 631ms	remaining: 2.35s
212:	learn: 0.0000041	total: 634ms	remaining: 2.34s
213:	learn: 0.0000040	total: 637ms	remaining: 2.34s
214:	learn: 

379:	learn: 0.0000006	total: 1.16s	remaining: 1.89s
380:	learn: 0.0000006	total: 1.16s	remaining: 1.89s
381:	learn: 0.0000006	total: 1.17s	remaining: 1.89s
382:	learn: 0.0000006	total: 1.17s	remaining: 1.88s
383:	learn: 0.0000006	total: 1.17s	remaining: 1.88s
384:	learn: 0.0000006	total: 1.18s	remaining: 1.88s
385:	learn: 0.0000006	total: 1.18s	remaining: 1.88s
386:	learn: 0.0000006	total: 1.18s	remaining: 1.87s
387:	learn: 0.0000006	total: 1.19s	remaining: 1.87s
388:	learn: 0.0000006	total: 1.19s	remaining: 1.87s
389:	learn: 0.0000006	total: 1.19s	remaining: 1.86s
390:	learn: 0.0000006	total: 1.19s	remaining: 1.86s
391:	learn: 0.0000006	total: 1.2s	remaining: 1.86s
392:	learn: 0.0000006	total: 1.2s	remaining: 1.85s
393:	learn: 0.0000006	total: 1.2s	remaining: 1.85s
394:	learn: 0.0000006	total: 1.21s	remaining: 1.85s
395:	learn: 0.0000006	total: 1.21s	remaining: 1.84s
396:	learn: 0.0000006	total: 1.21s	remaining: 1.84s
397:	learn: 0.0000006	total: 1.22s	remaining: 1.84s
398:	learn: 0.0

552:	learn: 0.0000003	total: 1.74s	remaining: 1.4s
553:	learn: 0.0000003	total: 1.74s	remaining: 1.4s
554:	learn: 0.0000003	total: 1.74s	remaining: 1.4s
555:	learn: 0.0000003	total: 1.75s	remaining: 1.39s
556:	learn: 0.0000003	total: 1.75s	remaining: 1.39s
557:	learn: 0.0000003	total: 1.75s	remaining: 1.39s
558:	learn: 0.0000003	total: 1.76s	remaining: 1.39s
559:	learn: 0.0000003	total: 1.76s	remaining: 1.38s
560:	learn: 0.0000003	total: 1.76s	remaining: 1.38s
561:	learn: 0.0000003	total: 1.77s	remaining: 1.38s
562:	learn: 0.0000003	total: 1.77s	remaining: 1.37s
563:	learn: 0.0000003	total: 1.77s	remaining: 1.37s
564:	learn: 0.0000003	total: 1.78s	remaining: 1.37s
565:	learn: 0.0000003	total: 1.78s	remaining: 1.36s
566:	learn: 0.0000003	total: 1.78s	remaining: 1.36s
567:	learn: 0.0000003	total: 1.79s	remaining: 1.36s
568:	learn: 0.0000003	total: 1.79s	remaining: 1.35s
569:	learn: 0.0000003	total: 1.79s	remaining: 1.35s
570:	learn: 0.0000003	total: 1.8s	remaining: 1.35s
571:	learn: 0.00

728:	learn: 0.0000002	total: 2.31s	remaining: 859ms
729:	learn: 0.0000002	total: 2.31s	remaining: 855ms
730:	learn: 0.0000002	total: 2.32s	remaining: 852ms
731:	learn: 0.0000002	total: 2.32s	remaining: 849ms
732:	learn: 0.0000002	total: 2.32s	remaining: 846ms
733:	learn: 0.0000002	total: 2.33s	remaining: 843ms
734:	learn: 0.0000002	total: 2.33s	remaining: 840ms
735:	learn: 0.0000002	total: 2.33s	remaining: 837ms
736:	learn: 0.0000002	total: 2.33s	remaining: 834ms
737:	learn: 0.0000002	total: 2.34s	remaining: 830ms
738:	learn: 0.0000002	total: 2.34s	remaining: 827ms
739:	learn: 0.0000002	total: 2.35s	remaining: 824ms
740:	learn: 0.0000002	total: 2.35s	remaining: 821ms
741:	learn: 0.0000002	total: 2.35s	remaining: 818ms
742:	learn: 0.0000002	total: 2.35s	remaining: 815ms
743:	learn: 0.0000002	total: 2.36s	remaining: 811ms
744:	learn: 0.0000002	total: 2.36s	remaining: 808ms
745:	learn: 0.0000002	total: 2.36s	remaining: 805ms
746:	learn: 0.0000002	total: 2.37s	remaining: 802ms
747:	learn: 

912:	learn: 0.0000001	total: 2.89s	remaining: 275ms
913:	learn: 0.0000001	total: 2.89s	remaining: 272ms
914:	learn: 0.0000001	total: 2.9s	remaining: 269ms
915:	learn: 0.0000001	total: 2.9s	remaining: 266ms
916:	learn: 0.0000001	total: 2.9s	remaining: 263ms
917:	learn: 0.0000001	total: 2.91s	remaining: 260ms
918:	learn: 0.0000001	total: 2.91s	remaining: 256ms
919:	learn: 0.0000001	total: 2.91s	remaining: 253ms
920:	learn: 0.0000001	total: 2.92s	remaining: 250ms
921:	learn: 0.0000001	total: 2.92s	remaining: 247ms
922:	learn: 0.0000001	total: 2.92s	remaining: 244ms
923:	learn: 0.0000001	total: 2.92s	remaining: 241ms
924:	learn: 0.0000001	total: 2.93s	remaining: 237ms
925:	learn: 0.0000001	total: 2.93s	remaining: 234ms
926:	learn: 0.0000001	total: 2.93s	remaining: 231ms
927:	learn: 0.0000001	total: 2.94s	remaining: 228ms
928:	learn: 0.0000001	total: 2.94s	remaining: 225ms
929:	learn: 0.0000001	total: 2.94s	remaining: 222ms
930:	learn: 0.0000001	total: 2.94s	remaining: 218ms
931:	learn: 0.0

In [40]:
test_id['class'] = preds
test_id

,id,class
0,TEST_000,0
1,TEST_001,1
2,TEST_002,2
3,TEST_003,1
4,TEST_004,0
...,...,...
170,TEST_170,1
171,TEST_171,2
172,TEST_172,2
173,TEST_173,1


In [41]:
# Decoding

for i, j in zip(range(3), ['A', 'B', 'C']):
    test_id['class'] = test_id['class'].replace(i, j)
    
test_id

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [42]:
test_id.to_csv('submission_04.csv', index=False, encoding='utf-8')